In [1]:
!nvidia-smi

Mon Aug 28 13:41:39 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   57C    P0              30W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from langchain.vectorstores import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
# from langchain.document_loaders import TextLoader
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import DirectoryLoader


# from InstructorEmbedding import INSTRUCTOR
# from langchain.embeddings import HuggingFaceInstructEmbeddings

In [3]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


2023-08-28 13:41:47.393528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 13:41:48.296073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


max_seq_length  512


# from_documents ids最好自己建立

In [56]:
# ids = [str(i) for i in range(1, len(docs) + 1)]
# example_db = Chroma.from_documents(docs, embedding_function, ids=ids)

# 如ids=null
#預設id生成會是像'1dba1291-4568-11ee-b90f-2cea7f594df2'

In [4]:
persist_directory = 'db'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

In [5]:
q1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [8]:
len(vectordb.similarity_search(q1))

4

In [7]:
vectordb.similarity_search(q1, k=1)

[Document(page_content='2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP  \nI. This guide will walk you through the process of requesting  RMA  through the \nWareconn Customer Portal.  \nII. Details on how to fill in the Part Number (PN) and Serial Number (SN)  \n1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1})]

In [10]:
from langchain.docstore.document import Document

new_doc =  Document(
    page_content="Wareconn is the best web platform for warranty maintenance.",
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

In [12]:
new_docs = [new_doc]

In [38]:
# add

id = vectordb.add_documents(
    new_docs,
    ids=["7"]
)
    

In [39]:
vectordb.similarity_search("What is the best web platform for warranty maintenance?", k=1)

[Document(page_content='Wareconn is the best web platform for warranty maintenance.', metadata={'source': 'wareconn.com', 'page': 1})]

In [40]:
vectordb._collection.count()

7

In [41]:
update_doc =  Document(
    page_content="Wareconn is the perfect web platform for warranty maintenance.",
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

In [45]:
# print(vectordb._collection.get(ids=['1dba1291-4568-11ee-b90f-2cea7f594df2']))

print(vectordb._collection.get(ids=id))

{'ids': ['7'], 'embeddings': None, 'documents': ['Wareconn is the best web platform for warranty maintenance.'], 'metadatas': [{'source': 'wareconn.com', 'page': 1}]}


In [46]:
vectordb.update_document(id[0], update_doc)

In [47]:
print(vectordb._collection.get(ids=id))

{'ids': ['7'], 'embeddings': None, 'documents': ['Wareconn is the perfect web platform for warranty maintenance.'], 'metadatas': [{'source': 'wareconn.com', 'page': 1}]}


In [48]:
vectordb._collection.delete(ids=id)

In [49]:
vectordb._collection.count()

6

In [50]:
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

In [51]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Mon Aug 28 14:09:23 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T

In [52]:
q1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"
ans1 = retriever.get_relevant_documents(q1)

In [53]:

for i, a in enumerate(ans1):
  print("-"*10+f"Answer {i+1}"+"-"*10)
  print(a.page_content)
  print(a.metadata)

----------Answer 1----------
2 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  
SOP  
I. This guide will walk you through the process of requesting  RMA  through the 
Wareconn Customer Portal.  
II. Details on how to fill in the Part Number (PN) and Serial Number (SN)  
1.1 RMA request  procedure (Parts return)  
1. System login  
⚫ Module: https://www.wareconn.com/  
⚫ Description:  Login  with account and password  
 
To log in, follow these steps:  
Go to https://www.wareconn.com/ and click Log in  
 
Fill in your email and password, then click Login
 
Click Enter  in the Customer portal
{'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1}
----------Answer 2----------
3 / 6 
This document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be 
used to intr